## Pseudo code for computing relative Mahlanobis distance

*Licensed under the Apache License, Version 2.0.*

To run this in a public Colab, change the GitHub link: replace github.com with [githubtocolab.com](http://githubtocolab.com)

<a href="https://githubtocolab.com/google/uncertainty-baselines/blob/main/experimental/ood_clm/Pseudo_code_for_computing_relative_Mahalanobis_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to compute the relative Mahalanobis distance (RMD) used in
[Out-of-Distribution Detection and Selective Generation for Conditional Language Models](https://arxiv.org/abs/2209.15558) for out-of-distributino detection for conditional language models.
The RMD score is shown to be a highly accurate and lightweight OOD detection method for CLMs, as demonstrated on abstractive summarization and translation.

In [ ]:
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt


import ood_utils  # local file import from baselines.jft

## Steps for computing Relative Mahalanobis distance (RMD) OOD score

In [ ]:
# (1) Prepare feature embeddings, embs_train_ind (NxD), for in-domain training data

# (2) Prepare the same number of feature embeddings, embs_train_ood (NxD), for
# general domain data (e.g. C4 for summarization, or ParaGrawl for translation).

# Here we create dummy values for feature embedings
N = 1000 # sample size
D = 256 # embedding dimension
mu_ind = np.zeros(D)
mu_ood = np.ones(D)*0.4
sigma = np.identity(D)
embs_train_ind = np.random.multivariate_normal(mu_ind, sigma, N)
embs_train_ood = np.random.multivariate_normal(mu_ood, sigma, N)

In [ ]:
# (3) Prepare feature embeddings, embs_ind and embs_ood for the test in-domain
# and test OOD data

# Here we create dummy values for feature embedings
embs_test_ind = np.random.multivariate_normal(mu_ind, sigma, N)
embs_test_ood = np.random.multivariate_normal(mu_ood, sigma, N)

In [ ]:
# (4) Fit Gaussian distributions for in-domain and general domain respectively
mean_list, cov = ood_utils.compute_mean_and_cov(embs_train_ind, np.zeros(N), np.zeros(1))
mean_list0, cov0 = ood_utils.compute_mean_and_cov(embs_train_ood, np.zeros(N), np.zeros(1))

# (5) Compute RMD OOD score for the test examples dist - dist_0

# Raw Mahalanobis distance
md_ind = ood_utils.compute_mahalanobis_distance(embs_test_ind, mean_list, cov).reshape(-1)
md_ood = ood_utils.compute_mahalanobis_distance(embs_test_ood, mean_list, cov).reshape(-1)

md0_ind = ood_utils.compute_mahalanobis_distance(embs_test_ind, mean_list0, cov0).reshape(-1)
md0_ood = ood_utils.compute_mahalanobis_distance(embs_test_ood, mean_list0, cov0).reshape(-1)

# Relative Mahalnobis distance
rmd_ind, rmd_ood = [md_ind - md0_ind, md_ood - md0_ood]

In [ ]:
# (6) Compute AUROC for OOD detection
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
plt_n = 0
scores = {'Mahalanobis distance': (md_ind, md_ood), 'Relative Mahalanobis distance': (rmd_ind, rmd_ood)}
for score_name, (scores_ind, scores_ood) in scores.items():
  labels_ind, labels_ood = [np.zeros_like(scores_ind), np.ones_like(scores_ood)]
  auc_rmd = ood_utils.compute_ood_metrics(np.concatenate((labels_ind, labels_ood)), np.concatenate((scores_ind, scores_ood)))
  print(f'OOD metrics based on {score_name}', auc_rmd)
  ax[plt_n].hist([scores_ind, scores_ood], label=('IND', 'OOD'), bins=20)
  auroc = auc_rmd['auroc']
  ax[plt_n].title.set_text(f'{score_name}: {auroc:.4f}')
  ax[plt_n].legend()
  plt_n += 1
plt.show()